# Data Preparation
### Data Exploration for Tuning a Foundation Model

**Project environment setup:**

- Load credentials and relevant Python Libraries
- If you were running this notebook locally, you would first install Vertex AI. In this classroom, this is already installed.

```
!pip install google-cloud-aiplatform
```
- You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

In [1]:
# !pip install utils
# !pip install google-cloud-aiplatform
# !pip install google-auth
# !pip install google-cloud-bigquery
# !pip install google-cloud-storage

In [4]:
from pathlib import Path
import os

path = Path("\\".join(os.path.join(os.getcwd()).split('\\')[:-1]))
os.chdir(path)

WindowsPath('C:/Users/abhmukherjee/Documents/Portfolio/GenerativeAI_Learning_Resources')

In [6]:
from utils import authenticate
credentials, PROJECT_ID = authenticate.gcp_authenticate() 

In [7]:
REGION = "us-central1"

In [8]:
import vertexai

In [9]:
vertexai.init(
    project = PROJECT_ID,
    location = REGION,
    credentials = credentials,
)

In [10]:
from google.cloud import bigquery

In [11]:
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials = credentials,
)

## Stack Overflow Public Dataset

- We will use [Stack Overflow Data](https://cloud.google.com/blog/topics/public-datasets/google-bigquery-public-datasets-now-include-stack-overflow-q-a) on BigQuery Public Datasets.
- The datasets include questions, answers and metadata related to Stack Overflow questions. Within this dataset, there are tables with data.
- Create a SQL query.

In [12]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

- The query is asking to retrieve `table_name` of all the `TABLES`
- Use the client to send our SQL and retrieve the data (tables names).

In [13]:
query_job = bq_client.query(QUERY_TABLES)

In [14]:
for row in query_job:
    for value in row.values():
        print(value)

posts_answers
users
posts_orphaned_tag_wiki
posts_tag_wiki
stackoverflow_posts
posts_questions
comments
posts_tag_wiki_excerpt
posts_wiki_placeholder
posts_privilege_wiki
post_history
badges
post_links
tags
votes
posts_moderator_nomination


## Data Retrieval

- We'll fetch some data from the data warehouse and store it in Pandas dataframe for visualization.
- Select all columns from  `posts_questions` and put the `LIMIT` as 3.

In [15]:
INSPECT_QUERY = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

In [16]:
import pandas as pd

In [17]:
query_job = bq_client.query(INSPECT_QUERY)

- Take the results of the query `-->` create an arrow table (which is part of [Apache Framework](https://arrow.apache.org/docs/index.html)) `-->` which goes into a Pandas dataframe.
- This allows for data to be in a format which is easier to read and explore with Pandas.

In [18]:
stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
stack_overflow_df.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,73210679,az acr login raises DOCKER_COMMAND_ERROR with ...,<p>Windows 11 with wsl2 ubuntu-22.04.</p>\n<p>...,73247188.0,1,0,NaT,2022-08-02 16:16:31.810000+00:00,NaN,2022-08-05 09:00:14.693000+00:00,2022-08-02 16:32:13.700000+00:00,None,11226740,None,11226740,None,1,0,azure-container-registry|docker-daemon,256
1,73284406,Run Azure log query from the command line with...,<p>I am trying to get the Azure log query data...,NaN,2,0,NaT,2022-08-08 21:53:16.703000+00:00,NaN,2022-08-10 04:49:20.527000+00:00,2022-08-09 08:12:13.920000+00:00,None,2395282,None,19123691,None,1,0,azure|azure-devops|azure-application-insights|...,256
2,73250763,Error CS0246: The type or namespace name 'Stre...,<p>I have these errors when trying to write th...,73251390.0,1,0,NaT,2022-08-05 13:43:25.850000+00:00,NaN,2022-08-05 16:40:00.610000+00:00,2022-08-05 16:40:00.610000+00:00,None,17654458,None,17654458,None,1,3,c#|unity3d,512


### Dealing with Large Datasets

- Large datasets for LLMs often don't fit into memory.
- Select all of the columns and rows of the table `posts_questions`.

In [19]:
QUERY_ALL = """
SELECT
    *
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
"""

In [20]:
query_job = bq_client.query(QUERY_ALL)

In [21]:
try:
    stack_overflow_df = query_job\
    .result()\
    .to_arrow()\
    .to_pandas()
except Exception as e:
    print('The DataFrame is too large to load into memory.', e)

The DataFrame is too large to load into memory. 403 Response too large to return. Consider specifying a destination table in your job configuration. For more details, see https://cloud.google.com/bigquery/troubleshooting-errors; reason: responseTooLarge, message: Response too large to return. Consider specifying a destination table in your job configuration. For more details, see https://cloud.google.com/bigquery/troubleshooting-errors

Location: US
Job ID: f16990bc-eaee-4d7f-892e-81d5a55006ba



**Note:** The data is too large to return, as it is not fitting into memory.

#### Joining Tables and Query Optimization

- When working with (large) data, query optimizing is needed in order to save time and resources.
- Select questions as `input_text` (column 1), answers as `output_text` (column 2).
- Take the questions from `posts_questions` and answers from `posts_answers`.
- Join the questions and their corresponding accepted answers based on their same `unique ID`.
- Making sure the question is about `Python`, and that it `has an answer`. And the date the question was posted is on or after `2020-01-01`
- Limit as 10,000

In [22]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) as input_text,
    a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [23]:
query_job = bq_client.query(QUERY)

In [24]:
### this may take some seconds to run
stack_overflow_df = query_job.result()\
                        .to_arrow()\
                        .to_pandas()

stack_overflow_df.head(2)

,input_text,output_text
0,RandomWalker Segmentation algorithm results in...,<p>Random walker only <em>expands</em> labels ...
1,How to get numpy.exp to work with an array as ...,<p>You have to encapsulate the denominator of ...


### Adding Instructions

- Instructions for LLMs have been shown to improve model performance and generalization to unseen tasks [(Google, 2022)](https://arxiv.org/pdf/2210.11416.pdf).
- Wihtout the instruction, it is only question and answer. Model might not understand what to do.
- With the instructions, the model gets a guideline as to what task to perform.

In [25]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stack overflow question on Python. \
Answer it like you are a developer answering Stack overflow questions.

Stack overflow question:
"""

- A new column will combine `INSTRUCTION_TEMPLATE` and the question `input_text`.
- This avoids overwritting of any existing column which might be needed.

In [26]:
stack_overflow_df['input_text_instruct'] = INSTRUCTION_TEMPLATE + ' ' + stack_overflow_df['input_text']

### Dataset for Tuning

- Divide the data into a training and evaluation. By default, 80/20 split is used.
- This (80/20 split) allows for more data to be used for tuni
- g. The evaluation split is used as unseen data during tuning to evaluate performance.
- The `random_state` parameter is used to ensure random sampling for a fair comparison. 

In [27]:
from sklearn.model_selection import train_test_split

train, evaluation = train_test_split(
    stack_overflow_df,
    ### test_size=0.2 means 20% for evaluation
    ### which then makes train set to be of 80%
    test_size=0.2,
    random_state=42
)

#### Different Datasets and Flow

- Versioning data is important.
- It allows for reproducibility, traceability, and maintainability of machine learning models.
- Get the timestamp.

In [35]:
import datetime

date = datetime.datetime.now().strftime("%H_%d_%m_%Y")

- Generate a `jsonl` file.
- Name it as `tune_data_stack_overflow_python_qa-{date}`

In [36]:
cols = ['input_text_instruct','output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)

In [37]:
training_data_filename = f"tune_data_stack_overflow_python_qa-{str(date)}.jsonl"

In [40]:
with open('GenAI_and_LLMOps/' + training_data_filename, "w") as f:
    f.write(tune_jsonl)

### Evaluation Set

In [41]:
cols = ['input_text_instruct','output_text']
tune_jsonl = evaluation[cols].to_json(orient="records", lines=True)

### change the file name
### use "tune_eval_data_stack_overflow_python_qa-{date}.jsonl"
evaluation_data_filename = f"tune_eval_data_stack_overflow_python_qa-{str(date)}.jsonl"

### write the file
with open('GenAI_and_LLMOps/' + evaluation_data_filename, "w") as f:
    f.write(tune_jsonl)